# Domain Definitions

In [54]:
import pandas as pd
from dataclasses import dataclass
from numpy.random import default_rng
from numpy.random import Generator
from pathlib import Path
from typing import Optional, List, Set, Dict, Tuple

In [34]:
@dataclass(frozen=True)
class Item:
    name: str
    value: float
    weight: int
    time_to_steal: int

    @property
    def tts(self):
        return self.time_to_steal

In [35]:
@dataclass(frozen=False)
class Node:
    name: str
    items: Set[Item]

    def __init__(self, name: str):
        self.name = name
        self.items = set()

    def add_item(self, name: str, value: float, weight, tts: int) -> 'Node':
        item = Item(name, value, weight, time_to_steal=tts)
        return self._add_item(item)

    def _add_item(self, item: Item) -> 'Node':
        self.items.add(item)
        return self

    def to_dict(self):
        return {'name': self.name, 'items': self.items}

    def to_dicts(self):
        return [{
            'name': self.name,
            'item': item.name,
            'value': item.value,
            'weight': item.weight,
            'tts': item.tts
        } for item in self.items]

    def __hash__(self) -> int:
        return self.name.__hash__()

In [36]:
@dataclass(frozen=True)
class Edge:
    source: Node
    destination: Node
    travelling_time: int
    travelling_cost: float

    @property
    def time(self):
        return self.travelling_time

    @property
    def cost(self):
        return self.travelling_cost

    def to_dict(self):
        return {
            'source': self.source.name,
            'destination': self.destination.name,
            'travelling_time': self.time,
            'travelling_cost': self.cost
        }

In [199]:
class Graph:
    nodes: Dict[str, Node]
    edges: Dict[Tuple[Node, Node], Edge]
    connections: Dict[Node, Set[Edge]]

    def __init__(self):
        self.nodes = dict()
        self.edges = dict()
        self.connections = dict()

    def get_node(self, name: str) -> Optional[Node]:
        return self.nodes.get(name)

    def get_all_nodes(self) -> List[Node]:
        return list(self.nodes.values())

    def get_all_edges(self) -> List[Edge]:
        return list(self.edges.values())

    def get_connections_from_node(self, node: Node) -> List[Edge]:
        return self.connections.get(node, {})

    def get_connections_from(self, node_name: str) -> List[Edge]:
        node: Optional[Node] = self.get_node(node_name)
        return self.get_connections_from_node(node)

    def find_connection_between(self, src: Node, dst: Node) -> Optional[Edge]:
        return self.edges.get((src, dst), None)

    def add_edge(self, src: str, dst: str, tt: int, tc: float) -> 'Graph':
        source = self.nodes.get(src, Node(src))
        destination = self.nodes.get(dst, Node(dst))
        edge = Edge(source, destination, travelling_time=tt, travelling_cost=tc)
        self._add_node(source)._add_node(destination)
        self.edges[(source, destination)] = edge
        self.connections[source] = self.connections.get(source, []) + [edge]
        return self

    def add_node(self, node_name: str) -> 'Graph':
        self.nodes[node_name] = Node(node_name)
        return self

    def _add_node(self, node: Node) -> 'Graph':
        self.nodes[node.name] = node
        return self

    def to_pandas_df(self):
        vertexes = [node_item for node in self.get_all_nodes() for node_item in node.to_dicts()]
        edges = [edge.to_dict() for edge in self.get_all_edges()]
        return (pd.DataFrame.from_records(vertexes), pd.DataFrame.from_records(edges))

# Graph Construction

In [200]:
datasets_dir = Path().absolute().parent.joinpath('datasets')
travelling_map = f"{datasets_dir}/travelling_map.csv"
treasure_map = f"{datasets_dir}/treasure_map.csv"

In [201]:
travels = pd.read_csv(travelling_map)
treasures = pd.read_csv(treasure_map)

In [202]:
g = Graph()

In [203]:
travels.apply(lambda row: g.add_edge(src=row['source'],
                                     dst=row['destination'],
                                     tt=row['travelling_time'],
                                     tc=row['travelling_cost'])
                           .add_edge(src=row['destination'],
                                     dst=row['source'],
                                     tt=row['travelling_time'],
                                     tc=row['travelling_cost']), axis=1)

treasures.apply(lambda row: g.get_node(row['city'])
                             .add_item(row['name'],
                                       row['value'],
                                       row['weight'],
                                       row['tts']), axis=1);

In [204]:
nodes, edges = g.to_pandas_df()

# Exploratory Analysis

In [205]:
nodes.head(3)

,name,item,value,weight,tts
0,Santa Paula,Coroa do Rei João II,10000,5,10
1,Campos,Espada sagrada,6500,6,5
2,Riacho de Fevereiro,Cálice do Santo Graal,7000,2,6


In [206]:
edges.head(3)

,source,destination,travelling_time,travelling_cost
0,Escondidos,Santa Paula,6,780
1,Santa Paula,Escondidos,6,780
2,Escondidos,Campos,5,350


# Problem Solving 

In [288]:
@dataclass(frozen=False)
class Individual:
    genes: List[Node]
    total_income: float
    travelling_cost: float
    traveling_time: int
    time_to_steal: int

    @property
    def profit(self):
        return self.total_income - self.travelling_cost

    @property
    def total_time(self):
        return self.traveling_time + self.time_to_steal

    @property
    def score(self):
        return self.profit/self.total_time


In [302]:
class GeneticAlgorithm:
    graph: Graph
    genes: Set[Node]
    rng: Generator

    def __init__(self, graph: Graph):
        self.graph = graph
        self.genes = list(graph.get_all_nodes())
        self.rng = default_rng()

    def generate_initial_batch(self, size: int = 10) -> List[Individual]:
        length = len(self.genes)
        return [self.generate_individual(length) for i in range(size)]

    def generate_individual(self, number_of_genes: int):
        genes = self.generate_individual_genes(number_of_genes)
        tuples: Tuple[Node, Node] = [(genes[idx], genes[num]) for idx, num in enumerate(range(1, len(genes)))]
        connections: List[Edge] = [self.graph.find_connection_between(src, dest) for src, dest in tuples]

        total_income: float = 0.0
        travelling_cost: float = 0.0
        travelling_time: int = 0
        time_to_steal: int = 0

        for c in connections:
            print(c)
            items = [(i.value, i.weight, i.tts) for i in c.destination.items]
            print(items)
            print("===========================================================================")


    def generate_individual_genes(self, number: int) -> List[Node]:
        indexes = self.rng.choice(number, size=number, replace=False)
        genes: List[Node] = [self.genes[idx] for idx in indexes]

        starting_point: Node = self.graph.get_node("Escondidos")
        endpoint_idx: int = (genes.index(starting_point))
        individual_genes: List[Node] = [starting_point] + genes[0:endpoint_idx+1]
        return individual_genes

    def fitness(self, indv: Individual):
        pass

    def mutation(self):
        pass


    def cross_over(self):
        pass

In [303]:
ga = GeneticAlgorithm(g)

In [304]:
individuals: List[Individual] = ga.generate_initial_batch(size=1)

Edge(source=Node(name='Escondidos', items=set()), destination=Node(name='Granada', items={Item(name='Fóssil da primeira galinha conhecida', value=2500, weight=2, time_to_steal=1)}), travelling_time=7, travelling_cost=413)
[(2500, 2, 1)]
Edge(source=Node(name='Granada', items={Item(name='Fóssil da primeira galinha conhecida', value=2500, weight=2, time_to_steal=1)}), destination=Node(name='Além-do-Mar', items={Item(name='Maior diamante do continente', value=5400, weight=2, time_to_steal=10)}), travelling_time=6, travelling_cost=582)
[(5400, 2, 10)]
Edge(source=Node(name='Além-do-Mar', items={Item(name='Maior diamante do continente', value=5400, weight=2, time_to_steal=10)}), destination=Node(name='Foz da Água Quente', items={Item(name='Quadro do maior pintor do século', value=2000, weight=4, time_to_steal=4)}), travelling_time=2, travelling_cost=292)
[(2000, 4, 4)]
Edge(source=Node(name='Foz da Água Quente', items={Item(name='Quadro do maior pintor do século', value=2000, weight=4, time